In [17]:
# Dependencies
import pandas as pd
import numpy as np
import requests
import json
import matplotlib.pyplot as plt
from pathlib import Path

# Idea to generate a random SNF to compare it to others in the category.
import random

In [18]:
url="https://data.cms.gov/provider-data/api/1/metastore/schemas/dataset/items/4pq5-n9py?show-reference-ids=false"

In [19]:
print(requests.get(url))

<Response [200]>


In [20]:
print(requests.get(url).json())

{'accessLevel': 'public', 'landingPage': 'https://data.cms.gov/provider-data/dataset/4pq5-n9py', 'bureauCode': ['009:38'], 'issued': '2023-08-01', '@type': 'dcat:Dataset', 'modified': '2024-04-01', 'released': '2024-04-24', 'keyword': [{'identifier': 'd03021cc-311c-5181-8eb6-7ccf7e76fc53', 'data': 'General Information'}, {'identifier': '203355e5-2454-505d-b356-08de15938ab9', 'data': 'Address'}, {'identifier': 'c59e48c7-3cc5-5a09-adec-a49493db72f6', 'data': 'Location'}, {'identifier': '70a212e5-bf1b-5cad-9ecd-150dab2bacf9', 'data': 'Ratings'}, {'identifier': '107b8826-e90b-5db3-ac30-ecdb99c73db6', 'data': 'Beds'}, {'identifier': '5692b1a6-1c82-55c4-9698-c7441733b985', 'data': 'Quality'}, {'identifier': 'a6f7b7c6-d26e-5f59-8a5b-c151bacb9907', 'data': 'Staffing'}, {'identifier': '35df7004-e437-5a8a-a96b-1bda5456ccca', 'data': 'Penalties'}], 'contactPoint': {'@type': 'vcard:Contact', 'fn': 'Nursing homes including rehab services', 'hasEmail': 'mailto:BetterCare@cms.hhs.gov'}, 'publisher': 

In [21]:
response = requests.get(url).json()
print(json.dumps(response, indent=4, sort_keys=True))

{
    "%modified": "2024-04-12T16:05:17-0400",
    "@type": "dcat:Dataset",
    "accessLevel": "public",
    "bureauCode": [
        "009:38"
    ],
    "contactPoint": {
        "@type": "vcard:Contact",
        "fn": "Nursing homes including rehab services",
        "hasEmail": "mailto:BetterCare@cms.hhs.gov"
    },
    "description": "General information on currently active nursing homes, including number of certified beds, quality measure scores, staffing and other information used in the Five-Star Rating System. Data are presented as one row per nursing home.",
    "distribution": [
        {
            "data": {
                "%Ref:downloadURL": [
                    {
                        "data": {
                            "checksum": null,
                            "filePath": "s3://pdc-s3-nhc-data/NH_ProviderInfo_Apr2024.csv",
                            "identifier": "5eba4f43478aeb4d0ab8d071e1e4655d",
                            "mimeType": "text/csv",
           

In [22]:
response = requests.get(url).json()
print(json.dumps(response["distribution"][0]["data"]["downloadURL"], indent=4, sort_keys=True))

"https://data.cms.gov/provider-data/sites/default/files/resources/5eba4f43478aeb4d0ab8d071e1e4655d_1712952317/NH_ProviderInfo_Apr2024.csv"


In [23]:
nursinghomedataURL = json.dumps(response["distribution"][0]["data"]['downloadURL'], indent=4, sort_keys=True)
print(nursinghomedataURL)

"https://data.cms.gov/provider-data/sites/default/files/resources/5eba4f43478aeb4d0ab8d071e1e4655d_1712952317/NH_ProviderInfo_Apr2024.csv"


In [24]:
cleaned_string = nursinghomedataURL.replace('"', ' ').strip()

In [25]:
cleaned_string

'https://data.cms.gov/provider-data/sites/default/files/resources/5eba4f43478aeb4d0ab8d071e1e4655d_1712952317/NH_ProviderInfo_Apr2024.csv'

In [36]:
# Define the full path to your Resources folder and the desired filename
path = Path("Resources")
filename = "NH_ProviderInfo_Apr2024.csv"

# Combine the path to the Resources folder with the desired filename
file_path = path / filename

# Ensure the directory exists (since it already should), but this will prevent errors
path.mkdir(parents=True, exist_ok=True)

# Download the file
response = requests.get(cleaned_string)
response.raise_for_status()  # Ensure that the download was successful

# Save the file
with open(file_path, 'wb') as file:
    file.write(response.content)

# Print the full file path where the file is saved
print(f"File successfully saved at {file_path}")

File successfully saved at Resources\NH_ProviderInfo_Apr2024.csv


In [35]:
# pd.read_csv(nursinghomedataURL).sample(5)
nursinghomedata=pd.read_csv(cleaned_string)
nursinghomedata.sample(5)

,CMS Certification Number (CCN),Provider Name,Provider Address,City/Town,State,ZIP Code,Telephone Number,Provider SSA County Code,County/Parish,Ownership Type,...,Number of Citations from Infection Control Inspections,Number of Fines,Total Amount of Fines in Dollars,Number of Payment Denials,Total Number of Penalties,Location,Latitude,Longitude,Geocoding Footnote,Processing Date
8201,325035,LA VIDA LLENA,10501 LAGRIMA DE ORO NE,ALBUQUERQUE,NM,87111,5052966700,0,Bernalillo,Non profit - Corporation,...,6.0,2,11427.91,0,2,"10501 LAGRIMA DE ORO NE,ALBUQUERQUE,NM,87111",35.1320,-106.522,NaN,2024-04-01
5511,215160,ELLICOTT CITY HEALTHCARE CENTER,3000 NORTH RIDGE ROAD,ELLICOTT CITY,MD,21043,4104617577,130,Howard,For profit - Corporation,...,11.0,2,115610.50,1,3,"3000 NORTH RIDGE ROAD,ELLICOTT CITY,MD,21043",39.2901,-76.820,NaN,2024-04-01
5067,185481,EMERALD TRACE,3802 TURKEYFOOT ROAD,ELSMERE,KY,41018,8593420200,580,Kenton,Non profit - Corporation,...,NaN,1,14335.00,0,1,"3802 TURKEYFOOT ROAD,ELSMERE,KY,41018",38.9835,-84.591,NaN,2024-04-01
4799,17E597,KIOWA HOSPITAL DISTRICT MANOR,1020 MAIN STREET,KIOWA,KS,67070,6208254117,30,Barber,Government - Hospital district,...,NaN,0,0.00,0,0,"1020 MAIN STREET,KIOWA,KS,67070",37.0170,-98.481,NaN,2024-04-01
11572,425297,CARLYLE SENIOR CARE OF WILLISTON LLC,5721 SPRINGFIELD HWY,WILLISTON,SC,29853,8032663229,50,Barnwell,For profit - Limited Liability company,...,0.0,1,12649.00,0,1,"5721 SPRINGFIELD HWY,WILLISTON,SC,29853",33.4102,-81.413,NaN,2024-04-01


In [11]:
# nursinghomedataREDUCED=nursinghomedata.drop(columns=['CMS Certification Number (CCN)', 'Provider SSA County Code', 'Geocoding Footnote', 'Processing Date'])
nursinghomedataREDUCED=nursinghomedata[[
        'Provider Name', 
        'City/Town',
        'State', 
        'Ownership Type', 
        'Number of Certified Beds', 
        'Average Number of Residents per Day', 
        'Number of Facility Reported Incidents', 
        'Number of Substantiated Complaints', 
        'Number of Citations from Infection Control Inspections', 
        'Number of Fines', 
        'Total Amount of Fines in Dollars',
        'Abuse Icon']]
    

nursinghomedataREDUCED.sample(25)

,Provider Name,City/Town,State,Ownership Type,Number of Certified Beds,Average Number of Residents per Day,Number of Facility Reported Incidents,Number of Substantiated Complaints,Number of Citations from Infection Control Inspections,Number of Fines,Total Amount of Fines in Dollars,Abuse Icon
12920,TAYLOR HEALTHCARE CENTER,GRAFTON,WV,For profit - Corporation,60,57.0,0,1,NaN,0,0.00,N
7418,MCKNIGHT PLACE EXTENDED CARE,SAINT LOUIS,MO,For profit - Corporation,70,62.8,1,1,NaN,5,8177.79,N
9217,LIBERTY COMMONS NSG & REHAB CTR OF ROWAN COUNTY,SALISBURY,NC,For profit - Limited Liability company,90,78.0,1,2,0.0,1,14521.00,Y
10676,OREGON VETERANS HOME,THE DALLES,OR,Non profit - Other,151,98.7,9,2,0.0,0,0.00,N
1085,CITY VIEW POST ACUTE,SAN FRANCISCO,CA,For profit - Limited Liability company,180,162.8,12,25,3.0,2,12772.50,N
4598,TOPEKA PRESBYTERIAN MANOR,TOPEKA,KS,Non profit - Corporation,68,61.2,2,0,NaN,1,657.80,Y
6534,THE ESTATES AT DELANO LLC,DELANO,MN,For profit - Corporation,39,31.3,0,1,NaN,2,19850.00,N
9581,SUNRISE POINTE NURSING & REHAB,MAPLE HEIGHTS,OH,For profit - Individual,99,88.7,0,8,1.0,1,22932.00,N
10051,LANFAIR CENTER FOR REHAB & NSG CARE INC,LANCASTER,OH,For profit - Corporation,84,77.3,0,10,0.0,1,5000.00,N
14292,REGENT CARE CENTER OAKWELL FARMS,SAN ANTONIO,TX,Government - Hospital district,194,75.7,1,8,3.0,1,10007.88,N


In [12]:
nursinghomedata.columns

Index(['CMS Certification Number (CCN)', 'Provider Name', 'Provider Address',
       'City/Town', 'State', 'ZIP Code', 'Telephone Number',
       'Provider SSA County Code', 'County/Parish', 'Ownership Type',
       'Number of Certified Beds', 'Average Number of Residents per Day',
       'Average Number of Residents per Day Footnote', 'Provider Type',
       'Provider Resides in Hospital', 'Legal Business Name',
       'Date First Approved to Provide Medicare and Medicaid Services',
       'Affiliated Entity Name', 'Affiliated Entity ID',
       'Continuing Care Retirement Community', 'Special Focus Status',
       'Abuse Icon', 'Most Recent Health Inspection More Than 2 Years Ago',
       'Provider Changed Ownership in Last 12 Months',
       'With a Resident and Family Council',
       'Automatic Sprinkler Systems in All Required Areas', 'Overall Rating',
       'Overall Rating Footnote', 'Health Inspection Rating',
       'Health Inspection Rating Footnote', 'QM Rating', 'QM Rating